In [1]:
import pandas as pd
import numpy as np
import redis
import csv

In [2]:
df = pd.read_csv('data/updatedcleandata.csv')
df_json = df.to_json(orient='records')
df.head()

,state,state_po,county_str,office,special,candidate,party,writein,candidatevotes,won,MOV16,MOV18,16to18swing
0,Arizona,AZ,apache_az,US Senator,False,Angela Green,green,False,961.0,False,0.368776,0.338343,-0.030434
1,Arizona,AZ,apache_az,US Senator,False,Kyrsten Sinema,democrat,False,16298.0,True,0.368776,0.338343,-0.030434
2,Arizona,AZ,apache_az,US Senator,False,Martha McSally,republican,False,7810.0,False,0.368776,0.338343,-0.030434
3,Arizona,AZ,apache_az,US Senator,False,NaN,NaN,True,18.0,False,0.368776,0.338343,-0.030434
4,Arizona,AZ,cochise_az,US Senator,False,Angela Green,green,False,1212.0,False,-0.225856,-0.209577,0.016279


In [8]:
df = df[df['writein']==False]

In [11]:
redis_host = 'localhost'
redis_port = 6379

r = redis.Redis(host=redis_host, port=redis_port,db=10)
r.flushdb()

True

In [14]:
with open('data/updatedcleandata.csv', newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        key = f"{row['candidate']}:{row['county_str']}"
        r.hset(key, mapping=row)

In [22]:
column_to_query = '16to18swing'
min_value = 0.3

# Scan all keys in Redis
cursor = 0
matched_keys = []

while True:
    cursor, keys = r.scan(cursor, match='*:*')

    for key in keys:
        value = r.hget(key, column_to_query)

        if value and float(value) > min_value:
            matched_keys.append(key)

    if cursor == 0:
        break

matched_values = []
for key in matched_keys:
    values = r.hgetall(key)

    decoded_values = {k.decode('utf-8'): v.decode('utf-8') for k, v in values.items()}

    matched_values.append(decoded_values)

In [17]:
state_d = dict()
cand_d = dict()
county_d = dict()

for value in matched_values:
    if value['candidate'] in cand_d:
        cand_d[value['candidate']] = cand_d[value['candidate']] + 1
    else:
        cand_d[value['candidate']] = 1
        
    if value['state'] in state_d:
        state_d[value['state']] = state_d[value['state']] + 1
    else:
        state_d[value['state']] = 1
        
    if value['county_str'] in county_d:
        county_d[value['county_str']] = county_d[value['county_str']] + 1
    else:
        county_d[value['county_str']] = 1

In [18]:
dict(sorted(state_d.items(), key=lambda item: item[1], reverse=True))

{'Minnesota': 165,
 'West Virginia': 153,
 'California': 114,
 'North Dakota': 33,
 'Montana': 27,
 'Ohio': 9,
 'Pennsylvania': 4,
 'Maine': 3,
 'New Mexico': 3}

In [19]:
column_to_query = '16to18swing'
min_value = -0.15

# Scan all keys in Redis
cursor = 0
matched_keys = []

while True:
    cursor, keys = r.scan(cursor, match='*:*')

    for key in keys:
        value = r.hget(key, column_to_query)

        if value and float(value) < min_value:
            matched_keys.append(key)

    if cursor == 0:
        break

matched_values = []
for key in matched_keys:
    values = r.hgetall(key)

    decoded_values = {k.decode('utf-8'): v.decode('utf-8') for k, v in values.items()}

    matched_values.append(decoded_values)

In [20]:
state_d = dict()
cand_d = dict()
county_d = dict()

for value in matched_values:
    if value['candidate'] in cand_d:
        cand_d[value['candidate']] = cand_d[value['candidate']] + 1
    else:
        cand_d[value['candidate']] = 1
        
    if value['state'] in state_d:
        state_d[value['state']] = state_d[value['state']] + 1
    else:
        state_d[value['state']] = 1
        
    if value['county_str'] in county_d:
        county_d[value['county_str']] = county_d[value['county_str']] + 1
    else:
        county_d[value['county_str']] = 1

In [21]:
dict(sorted(state_d.items(), key=lambda item: item[1], reverse=True))

{'Utah': 30, 'Mississippi': 12, 'Missouri': 11, 'Texas': 9}

In [24]:
keys

[b'Mitt Romney:davis_ut',
 b'Craig R. Bowden:kane_ut',
 b'Jenny Wilson:cache_ut',
 b'Tim Aalders:rich_ut',
 b':emery_ut',
 b':juab_ut',
 b'Mitt Romney:millard_ut',
 b':rich_ut',
 b'Tim Aalders:piute_ut',
 b'Craig R. Bowden:saltlake_ut',
 b'Jenny Wilson:saltlake_ut',
 b'Craig R. Bowden:garfield_ut',
 b'Jenny Wilson:wayne_ut',
 b'Jenny Wilson:grand_ut',
 b':uintah_ut',
 b'Craig R. Bowden:juab_ut',
 b'Tim Aalders:emery_ut',
 b'Tim Aalders:utah_ut',
 b'Jenny Wilson:emery_ut',
 b'Jenny Wilson:uintah_ut',
 b'Tim Aalders:tooele_ut',
 b':sanjuan_ut',
 b'Mitt Romney:boxelder_ut',
 b'Jenny Wilson:garfield_ut',
 b'Reed C. McCandless:millard_ut',
 b'Mitt Romney:beaver_ut',
 b'Craig R. Bowden:duchesne_ut',
 b':boxelder_ut',
 b'Mitt Romney:piute_ut',
 b'Reed C. McCandless:garfield_ut',
 b'Craig R. Bowden:sanjuan_ut',
 b':weber_ut',
 b':summit_ut',
 b'Tim Aalders:beaver_ut',
 b'Tim Aalders:garfield_ut',
 b'Reed C. McCandless:juab_ut',
 b'Jenny Wilson:davis_ut',
 b':wayne_ut',
 b'Jenny Wilson:beaver_u

In [28]:
pattern = '*:*_ut'

# Use the SCAN command to iterate over matching keys
cursor = 0
keys = []
while True:
    # Perform the SCAN command
    cursor, matching_keys = r.scan(cursor, match=pattern)

    # Add the matching keys to the result list
    keys.extend(matching_keys)
    # Stop if the cursor is back to 0, indicating the end of the iteration
    if cursor == 0:
        break

# Retrieve the values for the matching keys
values = r.mget(keys)

# Print the keys and their corresponding values
#for key, value in zip(keys, values):
#    print(key, value)

print(keys)

[k.decode('utf-8') for k in keys]

decoded_values = {k.decode('utf-8'): v.decode('utf-8') for k, v in values.items()}

[b'Mitt Romney:davis_ut', b'Craig R. Bowden:kane_ut', b'Jenny Wilson:cache_ut', b'Tim Aalders:rich_ut', b':emery_ut', b':juab_ut', b'Mitt Romney:millard_ut', b':rich_ut', b'Tim Aalders:piute_ut', b'Craig R. Bowden:saltlake_ut', b'Jenny Wilson:saltlake_ut', b'Craig R. Bowden:garfield_ut', b'Jenny Wilson:wayne_ut', b'Jenny Wilson:grand_ut', b':uintah_ut', b'Craig R. Bowden:juab_ut', b'Tim Aalders:emery_ut', b'Tim Aalders:utah_ut', b'Jenny Wilson:emery_ut', b'Jenny Wilson:uintah_ut', b'Tim Aalders:tooele_ut', b':sanjuan_ut', b'Mitt Romney:boxelder_ut', b'Jenny Wilson:garfield_ut', b'Reed C. McCandless:millard_ut', b'Mitt Romney:beaver_ut', b'Craig R. Bowden:duchesne_ut', b':boxelder_ut', b'Mitt Romney:piute_ut', b'Reed C. McCandless:garfield_ut', b'Craig R. Bowden:sanjuan_ut', b':weber_ut', b':summit_ut', b'Tim Aalders:beaver_ut', b'Tim Aalders:garfield_ut', b'Reed C. McCandless:juab_ut', b'Jenny Wilson:davis_ut', b':wayne_ut', b'Jenny Wilson:beaver_ut', b'Tim Aalders:uintah_ut', b':iron_